# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [11]:
# YOUR CHANGES HERE
import pandas as pd
# Read data
df = pd.read_csv("strawberry-prices.tsv", sep="\t")
df["month"] = pd.to_datetime(df["month"])

# Split train (2020-2023) and backtest target months (2024)
train = df[df["month"].dt.year <= 2023].copy()
test_2024 = df[df["month"].dt.year == 2024].copy()

# Add month-of-year
train["moy"] = train["month"].dt.month
test_2024["moy"] = test_2024["month"].dt.month

# Seasonal mean model: average price by month-of-year using 2020-2023
seasonal_means = train.groupby("moy")["price"].mean()

# Predict 2024 prices
backtest_pred = pd.DataFrame({
    "month": test_2024["month"].dt.strftime("%Y-%m-%d"),
    "price": test_2024["moy"].map(seasonal_means).values
})

# Save predictions
backtest_pred.to_csv("strawberry-backtest.tsv", sep="\t", index=False)
backtest_pred


,month,price
48,2024-01-01,4.362750
49,2024-02-01,4.091250
50,2024-03-01,3.688750
51,2024-04-01,3.787200
52,2024-05-01,3.526250
53,2024-06-01,3.275000
54,2024-07-01,3.194000
55,2024-08-01,3.489750
56,2024-09-01,3.581750
57,2024-10-01,3.926000


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [12]:
# YOUR CHANGES HERE
# Read ground truth and backtest predictions
prices = pd.read_csv("strawberry-prices.tsv", sep="\t")
backtest = pd.read_csv("strawberry-backtest.tsv", sep="\t")
prices["month"] = pd.to_datetime(prices["month"])
backtest["month"] = pd.to_datetime(backtest["month"])

# Filter ground truth to 2024
truth_2024 = prices[prices["month"].dt.year == 2024].copy()
# Merge predictions with ground truth
merged = truth_2024.merge(
    backtest,
    on="month",
    suffixes=("_actual", "_predicted")
)
# Compute residuals
residuals = merged["price_predicted"] - merged["price_actual"]
# Compute mean and standard deviation
mean_residual = residuals.mean()
std_residual = residuals.std()
# Save results
accuracy_df = pd.DataFrame({
    "mean": [mean_residual],
    "std": [std_residual]
})
accuracy_df.to_csv(
    "backtest-accuracy.tsv",
    sep="\t",
    index=False
)
accuracy_df


,mean,std
0,0.004856,0.274566


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [13]:
# YOUR CHANGES HERE
import pandas as pd
# Read full dataset
df = pd.read_csv("strawberry-prices.tsv", sep="\t")
df["month"] = pd.to_datetime(df["month"])
# Use all data from 2020 through 2024
train_full = df[df["month"].dt.year <= 2024].copy()
# Add month-of-year
train_full["moy"] = train_full["month"].dt.month
# Compute seasonal means 
seasonal_means = train_full.groupby("moy")["price"].mean()
# Create 2025 forecast months
forecast_months = pd.date_range(
    start="2025-01-01",
    end="2025-12-01",
    freq="MS"
)
forecast_df = pd.DataFrame({
    "month": forecast_months.strftime("%Y-%m-%d"),
    "price": [seasonal_means[m.month] for m in forecast_months]
})
# Save forecast
forecast_df.to_csv(
    "strawberry-forecast.tsv",
    sep="\t",
    index=False
)
forecast_df


,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840
5,2025-06-01,3.21540
6,2025-07-01,3.17840
7,2025-08-01,3.46120
8,2025-09-01,3.61380
9,2025-10-01,3.88440


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [16]:
# YOUR CHANGES HERE
from itertools import combinations
# Read 2025 forecast
forecast = pd.read_csv("strawberry-forecast.tsv", sep="\t")
forecast["month"] = pd.to_datetime(forecast["month"])
initial_capital = 1_000_000
freeze_cost = 0.20
storage_cost_per_month = 0.10
discount_factor = 0.9
results = []

# Iterate over all buy/sell month pairs (buy before sell)
for i, buy_row in forecast.iterrows():
    for j, sell_row in forecast.iterrows():
        if sell_row["month"] <= buy_row["month"]:
            continue
        buy_price = buy_row["price"]
        sell_price = sell_row["price"]
        # Months between buy and sell
        months_held = (
            (sell_row["month"].year - buy_row["month"].year) * 12
            + (sell_row["month"].month - buy_row["month"].month)
        )
        # Total cost per pint
        total_cost_per_pint = (
            buy_price
            + freeze_cost
            + storage_cost_per_month * months_held
        )

        # Revenue per pint
        revenue_per_pint = sell_price * discount_factor

        # Profit per pint
        profit_per_pint = revenue_per_pint - total_cost_per_pint

        # Skip unprofitable trades
        if profit_per_pint <= 0:
            continue

        # Max pints purchasable
        pints_purchased = int(initial_capital // total_cost_per_pint)

        expected_profit = pints_purchased * profit_per_pint

        results.append({
            "buy_month": buy_row["month"].strftime("%Y-%m-%d"),
            "sell_month": sell_row["month"].strftime("%Y-%m-%d"),
            "pints_purchased": pints_purchased,
            "expected_profit": expected_profit
        })

# Create results DataFrame
timings_df = pd.DataFrame(results)

# Sort by expected profit (descending)
timings_df = timings_df.sort_values("expected_profit", ascending=False)

# Save to TSV
timings_df.to_csv(
    "timings.tsv",
    sep="\t",
    index=False
)

timings_df.head()


,buy_month,sell_month,pints_purchased,expected_profit
3,2025-07-01,2025-12-01,257838,114385.703292
1,2025-06-01,2025-12-01,249041,76364.437994
5,2025-08-01,2025-12-01,246232,64225.677488
6,2025-09-01,2025-12-01,243084,50618.353656
2,2025-07-01,2025-11-01,264662,48634.289120


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [17]:
# YOUR CHANGES HERE


# Load timing results
timings = pd.read_csv("timings.tsv", sep="\t")

# Load backtest accuracy (mean and std of residuals)
accuracy = pd.read_csv("backtest-accuracy.tsv", sep="\t")

# Identify best profit scenario
best_row = timings.loc[timings["expected_profit"].idxmax()]
best_profit = best_row["expected_profit"]
pints_purchased = best_row["pints_purchased"]

# Backtest standard deviation of residuals
std_residual = accuracy["std"].iloc[0]

# One standard deviation impact on revenue
one_std_profit = pints_purchased * std_residual

# Save results
check_df = pd.DataFrame({
    "best_profit": [best_profit],
    "one_std_profit": [one_std_profit]
})

check_df.to_csv(
    "check.tsv",
    sep="\t",
    index=False
)

check_df


,best_profit,one_std_profit
0,114385.703292,70793.566481


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.